In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Basic RAG with Weaviate

Now - let's try performing RAG with the chunks that we've created. 

We will:
- Load & chunk a document
- Add the chunks to Weaviate, and generate vectors
- And perform RAG

We assume some familiarity with Weaviate here. 

(If not, check out the [Weaviate Quickstart](https://docs.weaviate.io/weaviate/quickstart), or ask questions in the live session!)

### Load and chunk a document

In [2]:
from pathlib import Path

def get_chunks_using_markers(src_text: str) -> list[str]:
    """
    Split the source text into chunks using markers.
    """
    marker = "\n##"

    # Split by marker and reconstruct with markers (except first chunk)
    parts = src_text.split(marker)
    chunks = []

    # Add first chunk if it exists and isn't empty
    if parts[0].strip():
        chunks.append(parts[0].strip())

    # Add remaining chunks with markers reattached
    for part in parts[1:]:
        if part.strip():
            chunks.append(marker + part.strip())

    return chunks


md_file = Path("data/parsed/hai_ai_index_report_2025_chapter_2-parsed-text.md")
md_text = md_file.read_text(encoding="utf-8")
chunk_texts = get_chunks_using_markers(md_text)

### Set up Weaviate

In [ ]:
import weaviate
import os

# Use Embedded Weaviate with:
# version: latest available (e.g. "1.32.0"), headers = {"X-Cohere-Api-Key": os.getenv("COHERE_API_KEY")}, env vars: {"LOG_LEVEL": "error"}
# BEGIN_SOLUTION
client = weaviate.connect_to_embedded(
    version="1.32.5",
    headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
    },
    environment_variables={
        "LOG_LEVEL": "error",  # Reduce amount of logs
        "ENABLE_API_BASED_MODULES": "true"
    }
)
# END_SOLUTION

### Set up a collection

In [4]:
client.collections.delete("Chunks")

{"action":"load_all_shards","build_git_commit":"08d409a988","build_go_version":"go1.25.0","build_image_tag":"HEAD","build_wv_version":"1.32.5","level":"error","msg":"failed to load all shards: context canceled","time":"2025-09-15T20:44:45+01:00"}


In [5]:
from weaviate.classes.config import Property, DataType, Configure, Tokenization

client.collections.create(
    name="Chunks",
    properties=[
        Property(
            name="document_title",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk_number",
            data_type=DataType.INT,
        ),
        Property(
            name="filename",
            data_type=DataType.TEXT,
            tokenization=Tokenization.FIELD
        ),
    ],
    vector_config=[
        # Add `Configure.Vectors.text2vec_cohere` vector to the collection with:
        # name: "default", source properties: ["document_title", "chunk"], and model: "embed-v4.0"
        # BEGIN_SOLUTION
        Configure.Vectors.text2vec_cohere(
            name="default",
            source_properties=["document_title", "chunk"],
            model="embed-v4.0"
        )
        # END_SOLUTION
    ],
    generative_config=Configure.Generative.cohere(
        model="command-r"
    )
)

### Import data

In [6]:
chunks = client.collections.get("Chunks")

In [7]:
from tqdm import tqdm

with chunks.batch.fixed_size(batch_size=100) as batch:
    for i, chunk_text in tqdm(enumerate(chunk_texts)):
        obj = {
            "document_title": "Stanford HAI Report 2025",
            "filename": "data/pdfs/hai_ai_index_report_2025_chapter_2.pdf",
            "chunk": chunk_text,
            "chunk_number": i + 1,
        }

        # Add object to batch for import with (batch.add_object())
        # BEGIN_SOLUTION
        batch.add_object(
            properties=obj
        )
        # END_SOLUTION

266it [00:00, 15875.99it/s]


### RAG queries



In [8]:
# Try a RAG query with:
# query (what to search for): "how to clean the washing machine" and
# grouped_task (prompt): "Briefly, what tasks do I need to perform to regularly maintain and clean the washing machine?"
# limit (how many objects to fetch): 10
# BEGIN_SOLUTION
response = chunks.generate.hybrid(
    query="Agentic AI use cases",
    limit=5,
    grouped_task="Briefly, what use cases and benefits of agentic AI are mentioned here?"
)
# END_SOLUTION

print("Query response:")
print(response.generative.text)

Query response:
The benefits and use cases of agentic AI mentioned in the text include:

- Agentic AI systems can assist with a diverse range of tasks, making them extremely versatile. These agents can help with academic research, scheduling, online shopping, and vacation booking.
- They have the potential to navigate both virtual and embodied environments, extending beyond reliance on linguistic commands alone.
- Agentic AI's ability to handle multimodality and interact with GUI and visual design tasks makes them valuable in many sectors.

However, the complexity of benchmarking agentic tasks, which often involve dynamic and varied situations, makes their development and evaluation challenging. There's a need for effective methods to assess their performance accurately.


### Recap - what's happening under the hood

![assets/llm_3_rag_weaviate.png](assets/llm_3_rag_weaviate.png)

We can review the passages:

In [9]:
print("Supporting passages:")
for o in response.objects:
    print(f"\n> Object: {o.uuid}:")
    print(o.properties['chunk'][:200]+"...")

Supporting passages:

> Object: 2d680701-09fd-46f4-a9df-803149b97d96:

##Gemini 2.0 in an agentic work fl ow

Source: AI Index, 2025

<!-- image -->...

> Object: 22ed7035-ede1-4542-b2e5-affb17c62ed3:

##2.8 AI Agents Chapter 2: Technical Performance

AI agents, autonomous or semiautonomous systems designed to operate within  speci fi c  environments to accomplish goals, represent an  exciting  fro...

> Object: 741a1a05-70a9-428d-8fb8-529ceddafc40:

##2.8 AI Agents

For decades, the topic of AI agents has been widely discussed in the AI community, yet few benchmarks have achieved widespread adoption, including those featured in last year's Index...

> Object: 6dc17834-1f10-4bd6-b5ab-a2dc2dfa91b6:

##2.8 AI Agents Chapter 2: Technical Performance

<!-- image -->

VAB presents a signi fi cant challenge for AI systems. The topperforming model, GPT-4o, achieves an overall success rate of just 36.2...

> Object: dfa91c2b-2ab9-4db2-9261-79a62606fe66:

##VisualAgentBench

VisualAgentBench  (

In [10]:
client.close()

{"build_git_commit":"08d409a988","build_go_version":"go1.25.0","build_image_tag":"HEAD","build_wv_version":"1.32.5","error":"cannot find peer","level":"error","msg":"transferring leadership","time":"2025-09-15T20:45:00+01:00"}
